### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5146,2023-10-20,Eua Nba,20:00,Indiana Pacers,Cleveland Cavaliers,2.10,1.77,224.5,1.91,1.91,1.5,1.91,1.91
5147,2023-10-20,Eua Nba,20:00,Philadelphia 76ers,Atlanta Hawks,1.80,2.05,220.5,1.91,1.91,-2.5,1.91,1.91
5148,2023-10-20,Eua Nba,20:30,Toronto Raptors,Washington Wizards,1.45,2.80,219.5,1.91,1.91,-5.5,1.91,1.91
5149,2023-10-20,Eua Nba,21:00,Dallas Mavericks,Detroit Pistons,1.50,2.70,223.5,1.91,1.91,-5.5,1.91,1.91
5150,2023-10-20,Eua Nba,21:00,Houston Rockets,Miami Heat,1.69,2.25,219.5,1.91,1.91,-2.5,1.91,1.91
5151,2023-10-20,Eua Nba,21:00,Milwaukee Bucks,Memphis Grizzlies,1.57,2.55,223.5,1.91,1.91,-4.5,1.91,1.91
5152,2023-10-20,Eua Nba,23:00,Golden State Warriors,San Antonio Spurs,1.45,2.80,226.5,1.91,1.91,-5.5,1.91,1.91
5153,2023-10-20,Argentina Liga A,21:00,Argentino,Obras,2.30,1.56,157.5,1.83,1.83,3.5,1.83,1.83
5154,2023-10-20,Argentina Liga A,22:00,Independiente de Oliva,San Martin,2.00,1.71,159.5,1.83,1.83,1.5,1.83,1.83
5155,2023-10-20,Japão B.League,23:55,Hiroshima D.,Utsunomiya Brex,2.30,1.56,146.5,1.83,1.83,1.5,1.83,1.83


## Apostas do dia

### Over v1 (LDA)

In [11]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Indiana Pacers,Cleveland Cavaliers,224.5,1.91,0.7846,Over
1,20:00,Eua Nba,Philadelphia 76ers,Atlanta Hawks,220.5,1.91,0.8018,Over
2,20:30,Eua Nba,Toronto Raptors,Washington Wizards,219.5,1.91,0.8004,Over
3,21:00,Eua Nba,Dallas Mavericks,Detroit Pistons,223.5,1.91,0.7892,Over
4,21:00,Eua Nba,Houston Rockets,Miami Heat,219.5,1.91,0.7859,Over
5,21:00,Eua Nba,Milwaukee Bucks,Memphis Grizzlies,223.5,1.91,0.7777,Over
6,23:00,Eua Nba,Golden State Warriors,San Antonio Spurs,226.5,1.91,0.7945,Over


### Over v1 (LDA)

In [12]:
# Duplicando o dataset
df_over_v1 = jogos.copy()

df_over_v1.reset_index(drop=True, inplace=True)
df_over_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v1.empty:
    mdl_over_v1 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v1, df_over_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Over v2 (LDA)

In [13]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Over v3 (Blend)

In [14]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['HA_Line'] < -8.5) | (prev['HA_Line'] > -5.5))]
    prev = prev[((prev['CV_ML'] >= 0) & (prev['CV_ML'] < 0.65)) | ((prev['CV_ML'] >= 0.77) & (prev['CV_ML'] < 0.87)) | ((prev['CV_ML'] >= 0.95) & (prev['CV_ML'] < 1.5))]


    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (BLEND) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
